In [70]:
import sys, os
sys.path.append(os.path.abspath('..'))

In [35]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layer

import numpy as np

In [71]:
import cgael

In [11]:
chat = cgael.LanguageTokenSet("CHAT", '-')
denoise = cgael.LanguageDenoiseLayer()

a = chat.encode("CATCH THAT CAT", (3,5))
print("Encoded:", a)

b = denoise(a)
print("Denoised:", b)

c = chat.decode(b)
print("Decoded:", c)

Encoded: tf.Tensor(
[[1 3 4 1 2]
 [4 2 3 4 0]
 [1 3 4 0 0]], shape=(3, 5), dtype=int64)
Denoised: tf.Tensor(
[[1 3 4 1 2]
 [4 2 3 4 0]
 [1 3 4 0 0]], shape=(3, 5), dtype=int64)
Decoded: CATCH THAT CAT


In [6]:
from cgael.helpers import ischar

ischar(8)

False

In [7]:
import random

In [15]:
def random_color(blur=0.0):
    return random.random()

random_color()

0.5479491076734134

In [16]:
random.random()

0.3537997256763976

In [17]:
import math
math.lerp(3,2,.4)

AttributeError: module 'math' has no attribute 'lerp'

In [22]:
import numpy as np
np.interp([3,2],.4)

TypeError: interp() missing 1 required positional argument: 'fp'

In [ ]:
def lerp(start, end, t):
    return start + t * (end - start)

In [23]:
def lerp(a, b, t):
    """
    A simple linear interpolation implementation. 
    Interpolates between a and b by t.
    """
    return (b - a) * t + a

In [27]:
lerp(2, 5, .4)

3.2

In [74]:
x = random.random()
lerp(x, round(x), .9)

0.02020608532451898

In [ ]:
def random_blur():
    """
    Returns a random number between 0 and 1 pulled to 
    """

In [5]:
from enum import Enum

In [6]:
class Swatch(Enum):
    WHITE = 1
    BLACK = 2
    RED = 3
    GREEN = 4
    BLUE = 5
    YELLOW = 6
    CYAN = 7
    MAGENTA = 8

In [7]:
def sample_swatch(swatch:Swatch, blurriness=0):
    """
    Samples a Swatch using the BGR colorspace.

    Parameters
    ---
    swatch : Swatch
        The swatch to sample from.
    blurriness : float
        A real number on the range [0, 1] which indicates how "blurry" a single value can be.
        If 0, only 0's and 1's will be returned.
        A higher blurriness value will allow values to be returned closer to 0.5.
    """
    def sample_low():
        return random.random() * blurriness * .5
    def sample_high():
        return 1-sample_low()
    match swatch:
        case Swatch.WHITE:
            return (sample_high(), sample_high(), sample_high())
        case Swatch.BLACK:
            return (sample_low(), sample_low(), sample_low())
        case Swatch.RED:
            return (sample_low(), sample_low(), sample_high())
        case Swatch.GREEN:
            return (sample_low(), sample_high(), sample_low())
        case Swatch.BLUE:
            return (sample_high(), sample_low(), sample_low())
        case Swatch.YELLOW:
            return (sample_low(), sample_high(), sample_high())
        case Swatch.CYAN:
            return (sample_high(), sample_high(), sample_low())
        case Swatch.MAGENTA:
            return (sample_high(), sample_low(), sample_high())
        case _:
            raise ValueError(f"Could not sample swatch '{swatch}'.")

In [8]:
sample_swatch(Swatch.RED, 1)

(0.4313116759856827, 0.03076021084778391, 0.6642806670473054)

In [1]:
import pygad

In [ ]:
class GeneratorSampleMethod(Enum):
    

In [59]:
class SimpleColorGenerator():
    def __init__(self, swatches:list, blur:float, fixed:bool, batch_size:int=None):
        """
        Parameters
        ---
        swatches : list(Swatch)
            The swatches to sample from.
        blur : float
            The blurriness value to sample with.
        fixed : bool
            If True, each batch will be consist of 1 sample per swatch.
            If False, each batch will consist of 'batch_size' random samples from random swatches.
        batch_size : int
            How big a batch size should be.
            Will be ignored if 'fixed' is set to True.
        """
        self.swatches = swatches
        self.blur = blur
        self.fixed = fixed
        self.batch_size = len(self.swatches) if batch_size is None else batch_size

        self.new_batch()

    def new_batch(self):
        """
        Generates a random new batch.
        """
        if self.fixed:
            x = [x for x in self.swatches] # deepcopy.
            random.shuffle(x)
        else:
            x = random.choices(self.swatches, k=self.batch_size)
        self.current_batch = np.array([sample_swatch(y, self.blur) for y in x])
        return self.current_batch

    def __call__(self):
        return self.current_batch

In [9]:
import random

In [64]:
gen = SimpleColorGenerator([Swatch.RED, Swatch.GREEN], 0, True, 4)
gen()

array([[0., 1., 0.],
       [0., 0., 1.]])

In [67]:
random.choices([Swatch.RED, Swatch.GREEN, Swatch.BLUE], k=2)

[<Swatch.GREEN: 4>, <Swatch.BLUE: 5>]

In [74]:
class SimpleColorModel():
    def __init__(self, token_set:cgael.LanguageTokenSet, word_count:int, word_length:int, color_count:int=1, color_channels:int=3, listener_embedding_size:int=4):
        self.token_set = token_set
        self.word_count = word_count
        self.word_length = word_length
        self.color_count = color_count
        self.color_channels = color_channels

        self.speaker = self._build_speaker()
        self.listener = self._build_listener(listener_embedding_size)
        self.model = self._build_model(self.speaker, self.listener)

    def _build_speaker(self):
        x = y = layer.Input((self.color_count, self.color_channels))
        y = layer.Flatten()(y)
        y = layer.Dense(self.word_count * self.word_length * self.token_set.token_count, activation="sigmoid")(y)
        y = layer.Reshape((self.word_count, self.word_length, self.token_set.token_count))(y)

        y = cgael.layers.ArgmaxLayer()(y)
        y = cgael.layers.LanguageDenoiseLayer()(y)
        
        return keras.Model(x, y)
    
    def _build_listener(self, embedding_size):
        x = y = layer.Input((self.word_count, self.word_length))
        y = layer.Embedding(self.token_set.token_count, embedding_size, embeddings_initializer="random_normal")(y)
        y = layer.Flatten()(y)
        y = layer.Dense((self.color_count * self.color_channels), activation="sigmoid")(y)
        y = layer.Reshape((self.color_count, self.color_channels))(y)

        return keras.Model(x, y)
    
    def _build_model(self, speaker, listener):
        x = y = layer.Input((self.color_count, self.color_channels))
        y = z = speaker(y)
        y = listener(y)
        
        return keras.Model(x, [z, y])
    
    #def _fitness(self, ga_instance, solution, sol_idx):
    #    pred = pygad.kerasga.predict(model=self.model, solution=solution, data=)

token_set = cgael.LanguageTokenSet("XO", '-')
model = SimpleColorModel(token_set=token_set, word_count=1, word_length=3, color_count=1, color_channels=3, listener_embedding_size=4)
model.model.summary()

AttributeError: 'LanguageTokenSet' object has no attribute 'encoder'